# Exploring BiblHertz Data

In [15]:
import httpx, os, sys, json
from rich import print
from pyld import jsonld
import pyoxigraph as px

In [3]:
r = httpx.get("https://gn.biblhertz.it/fotothek/seo", timeout=60)
if r.status_code == 200:
    open("biblhertz.json", "wb").write(r.content)
    DATA = r.json()

In [21]:
# Convert the JSON-LD document to RDF
print("Loading JSONLD doc")
DATA_quads = jsonld.normalize(DATA, {'format':'application/n-quads'})
open("biblhertz.nt", "w").write(DATA_quads)
print("Loading triples into store")
store = px.Store()
store.bulk_load(DATA_quads.encode("utf8"), "application/n-quads")
print(len(store))
store.dump("biblhertz.ttl", "text/turtle")

Loading JSONLD doc

Loading triples into store

3446

At this point we want to explore what the provided triples look like,but it is unwieldy doing that visually browsing the .ttl as a text file on disk.

But we can fire up a shmarql to click around, in the same dir as we just saved the `biblhertz.ttl` file, do:

```shell
docker run --rm -p 8000:8000 -it -v $(pwd):/data -e DATA_LOAD_PATHS=/data  ghcr.io/epoz/shmarql:latest
```

And then [open this in your browser](http://localhost:8000/shmarql)

In [23]:
imageObjects = [s for s,p,o, _ in store.quads_for_pattern(None, px.NamedNode("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), px.NamedNode("http://schema.org/ImageObject"))]
print(len(imageObjects))
    

96

There are only 96 schema:ImageObject items in this week, and a lot of blank nodes.

Most of the [schema:contentUrl images](http://localhost:8000/shmarql?p=%3Chttp%3A//schema.org/contentUrl%3E&o=%3Fo) give a 404 

There are [a lot of blanknodes](http://localhost:8000/shmarql) generated by the JSONLD, can we filter them?

Some of the schema:keywords IRIs lead to a 404, but [the used item shows some triples](http://localhost:8000/shmarql?e=http://127.0.0.1:8000/sparql&p=%3Fp&s=%3Chttps%3A//www.degruyter.com/database/AKL/entry/_/html%3E) .